In [ ]:
# 以下所有代码只能在聚宽的研究环境下运行
import json
import traceback
import numpy as np
import pandas as pd
from jqdata import opt
from pathlib import Path

In [ ]:
def get_mission_list(count: int, num_per_group: int):
    group_num, remind = divmod(count, num_per_group)
    body = np.arange(group_num * num_per_group).reshape((group_num, num_per_group)).tolist()
    tail = np.arange(group_num * num_per_group, count).tolist()
    if tail:
        body.append(tail)
    return body


def get_option_basic():
    option_q = query(opt.OPT_CONTRACT_INFO).filter(opt.OPT_CONTRACT_INFO.underlying_symbol=='510050.XSHG')
    return opt.run_query(option_q)
        
        
def is_file_exist(filename: str):
    filepath = Path.cwd().joinpath(filename)
    return filepath.exists()


def get_bar_by_code(code: str):
    price_q = query(opt.OPT_DAILY_PRICE).filter(opt.OPT_DAILY_PRICE.code==code)
    return opt.run_query(price_q)
    

def get_bar_in_sub_list(option_basic: pd.DataFrame, sub_mission_list: list):
    df_list = []
    is_all_delist = True
    for mission_id in sub_mission_list:
        contract = option_basic.iloc[mission_id]
        code = contract['code']
        name = contract['name']
        trading_code = contract['trading_code']
        
        df = get_bar_by_code(code)
        df['trading_code'] = trading_code
        df['name'] = name
        status = contract['contract_status']
        df['status'] = status
        if status == 'LIST':
            is_all_delist = False
        df_list.append(df)
#         print(f"{code}-{trading_code}-{name}数据获取成功")
    group_df = pd.concat(df_list, axis=0)
    return group_df, is_all_delist

In [ ]:
# 获取数据
num_per_group = 100
option_basic = get_option_basic()
all_mission_list = get_mission_list(len(option_basic), num_per_group)

for idx, sub_mission_list in enumerate(all_mission_list):
    filename = "option_daily_bar_archived_{:0>4d}.csv".format(idx)
    if not is_file_exist(filename):
        try:
            df, is_all_delist = get_bar_in_sub_list(option_basic, sub_mission_list)
        except:
            print(f"获取单组数据出错触发异常：\n{traceback.format_exc()}")
        else:
            if len(sub_mission_list) == num_per_group and is_all_delist:
                df.to_csv(filename)
            else:
                filename = filename = "option_daily_bar_unarchived_{:0>4d}.csv".format(idx)
                df.to_csv(filename)
            print(f"任务ID系列：{sub_mission_list}")
            print("单组数据保存成功！")
            print("=" * 50)
    else:
        print(f"{filename}已归档无需下载")
        pass